In [1]:
pip install ipykernel

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mysql-connector-python

  Using cached mysql_connector_python-9.4.0-cp313-cp313-win_amd64.whl.metadata (7.7 kB)
Using cached mysql_connector_python-9.4.0-cp313-cp313-win_amd64.whl (16.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import mysql.connector as db
from datetime import datetime

In [3]:
#Read data from dataset
file_path="traffic_stops - traffic_stops_with_vehicle_number.csv"
df=pd.read_csv(file_path)
df.head()

C:\Users\Navee\AppData\Local\Temp\ipykernel_11108\3383697650.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [4]:
#Handle null values
df.dropna(axis=1,how='all',inplace=True)
#fill null values with suitable values
df.fillna({
    'driver_age':df['driver_age'].median,
    'search_type':'None',
    'stop_duration':'Unknown',
    'violation':'Unknown',
    'stop_outcome':'Unknown',
    
},inplace=True)
#convert datetime to timestamp
df['time_stamp']=pd.to_datetime(df['stop_date']+' '+ df['stop_time'])
df.head()

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,time_stamp
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473,2020-01-01 00:00:00
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441,2020-01-01 00:01:00
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264,2020-01-01 00:02:00
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807,2020-01-01 00:03:00
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305,2020-01-01 00:04:00


In [7]:
import pymysql
connection=pymysql.connect(
    host="localhost",
    user="naveen v",
    password="Naveennavi@07"
    )
curr=connection.cursor()

In [5]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [ ]:
curr.execute("CREATE DATABASE IF NOT EXISTS trafficpostlogs_db;")

1

In [9]:
curr.execute("USE trafficpostlogs_db;")

0

In [10]:
duration_map={
    '0-15 Min':7.5 ,#midpoint of 0-15
    '16-30 Min':23,#midpoint of 16-30
    '30+ Min':45,#assume ~45 min for 30+
}
df['stop_duration_num']=df['stop_duration'].map(duration_map)

In [11]:
df['stop_duration_num']

0        23.0
1        23.0
2        23.0
3         7.5
4        23.0
         ... 
65533    23.0
65534    23.0
65535    45.0
65536     7.5
65537    23.0
Name: stop_duration_num, Length: 65538, dtype: float64

In [12]:
curr.execute("USE trafficpostlogs_db;")
curr.execute("""
    CREATE TABLE IF NOT EXISTS trafficpostlogs_data (
        stop_date DATE,
        stop_time TIME,
        country_name VARCHAR(50),
        driver_gender VARCHAR(50),
        driver_age_raw INT,
        driver_age INT,
        driver_race VARCHAR(50),
        violation_raw VARCHAR(50),
        violation VARCHAR(50),
        search_conducted BOOLEAN,
        search_type VARCHAR(50),
        stop_outcome VARCHAR(50),
        is_arrested BOOLEAN,
        stop_duration VARCHAR(50),
        drugs_related_stop BOOLEAN,
        vehicle_number VARCHAR(50),
        time_stamp DATETIME,
        stop_duration_num INT
    );
""")

0

In [14]:
insertQuery="""INSERT INTO trafficpostlogs_db.trafficpostlogs_data(
stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,
search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,time_stamp,stop_duration_num)
VALUES
(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""
for _, row in df.iterrows():
    curr.execute(insertQuery,(
        row['stop_date'],
        row['stop_time'],
        row.get('country_name'),
        row.get('driver_gender'),
        row.get('driver_age_raw'),
        int(row.get('driver_age')) if row.get('driver_age')is not None else None,
        row.get('driver_race'),
        row.get('violation_raw'),
        row.get('violation'),
        bool(row.get('search_conducted'))if row.get('search_conducted')is not None else None,
        row.get('search_type'),
        row.get('stop_outcome'),
       bool(row.get('is_arrested'))if row.get('is_arrested')is not None else None,
       row.get('stop_duration'),
       bool(row.get('drugs_related_stop'))if row.get('drugs_related_stop')is not None else None,
       row.get('vehicle_number'),
       row['time_stamp'],
       int(row.get('stop_duration_num')) if row.get('stop_duration_num')is not None else None,
    
 ))
  
    connection.commit()

In [15]:
pip install plotly

   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.8 MB 6.5 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.8 MB 4.8 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.8 MB 4.2 MB/s eta 0:00:02
   ------------ --------------------------- 3.1/9.8 MB 4.4 MB/s eta 0:00:02
   ----------------- ---------------------- 4.2/9.8 MB 4.2 MB/s eta 0:00:02
   -------------------- ------------------- 5.0/9.8 MB 4.2 MB/s eta 0:00:02
   ----------------------- ---------------- 5.8/9.8 MB 4.1 MB/s eta 0:00:01
   --------------------------- ------------ 6.8/9.8 MB 4.1 MB/s eta 0:00:01
   ------------------------------- -------- 7.6/9.8 MB 4.1 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.8 MB 4.0 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.8 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 4.0 MB/s  0:00:02
Note: you may need to 